In [0]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("FraudDetection").getOrCreate()

# Load the cleaned_data table from the Databricks catalog
df = spark.sql("SELECT * FROM real_time_fraud_detection.default.cleaned_data WHERE isFraud = 1")

# Convert to Pandas for easier API processing
fraud_cases = df.toPandas()

display(fraud_cases)

In [0]:
import requests
import json

# ServiceNow API Credentials
SNOW_INSTANCE = "https://dev183227.service-now.com/"
SNOW_USER = "admin"
SNOW_PASSWORD = "dB8TWzoF!i0$"
SNOW_TABLE_API = "/api/now/table/incident"

# Function to create an incident in ServiceNow
def create_servicenow_incident(user, card, txn_id, addr, amount):
    short_desc = f"🚨 Fraud Alert: Suspicious Transaction {txn_id} Detected"

    description = (
        f"User **{user} (Card: {card})** made a transaction **(Transaction ID: {txn_id})** "
        f"associated with **Card: {card}** to merchant **{addr}**.\n"
        f"This transaction has been flagged as **potential fraud** due to unusual activity.\n\n"
        f"A fraud investigation ticket has been opened for further analysis."
    )
    
    url = f"{SNOW_INSTANCE}{SNOW_TABLE_API}"
    headers = {"Content-Type": "application/json"}
    data = {
        "short_description": short_desc,
        "description": description,
        "urgency": "1",  # High priority
        "impact": "1",  # High impact
        "category": "Fraud Detection",
        "subcategory": "Transaction Fraud"
    }

    response = requests.post(url, auth=(SNOW_USER, SNOW_PASSWORD), headers=headers, data=json.dumps(data))
    
    if response.status_code in [200, 201]:
        print(f"✅ Incident created for Transaction {txn_id}: {response.json()}")
    else:
        print(f"❌ Error creating incident for Transaction {txn_id}: {response.status_code}, {response.text}")


In [0]:
for index, row in fraud_cases.iterrows():
    create_servicenow_incident(
        user=row["card1"],
        card=row["card1"],
        txn_id=row["TransactionID"],
        addr=row["addr1"],
        amount=row["TransactionAmt"]
    )